In [121]:
from datetime import datetime
import pandas as pd
import numpy as np

In [122]:
podatkovna_baza_kart = pd.read_csv("specificne_lastnosti_kart.csv")
pojavitve_barve = podatkovna_baza_kart.groupby("barva").size()
verjetnosti_barve = pojavitve_barve / len(podatkovna_baza_kart)

verjetnosti_barve

barva
b        0.1465
bg       0.0120
bgu      0.0010
br       0.0145
brg      0.0005
c        0.1505
g        0.1390
gu       0.0165
gur      0.0010
gw       0.0150
gwu      0.0005
r        0.1445
rg       0.0150
rgw      0.0005
ru       0.0010
rw       0.0095
rwb      0.0010
u        0.1330
ub       0.0095
ubr      0.0015
ug       0.0010
ur       0.0100
urw      0.0005
uw       0.0005
w        0.1445
wb       0.0135
wbg      0.0005
wu       0.0145
wub      0.0005
wubrg    0.0020
dtype: float64

In [123]:
def odstrani_konce(beseda, konci):
    for konec in konci:
        if beseda.endswith(konec):
            return beseda[:-len(konec)]
    return beseda

# Tukaj bi bilo morda vredno poizkusiti brez korenov besed. Namreč v resnici pri magicu ni tako zelo veliko besed, zato korenjenje morda sploh ni potrebno
# Naredimo lahko funkcijo s koreni in brez

konci_ki_jih_bomo_odstranili = ["ing", "ed", "en", "s"]
 
def koren_besede(beseda, ne_naredi_korenov = False):
    beseda = ''.join(znak for znak in beseda) #  if znak.isalpha()
    if not beseda:
        return '$'
    if not ne_naredi_korenov:
        beseda = odstrani_konce(beseda, konci_ki_jih_bomo_odstranili)
        beseda = beseda.rstrip('aeiou')
    if not beseda:
        return '@'
    return beseda

def koreni_besed(niz, ne_naredi_korenov = False):
    return pd.Series(sorted({
        koren_besede(beseda, ne_naredi_korenov=ne_naredi_korenov) for beseda in str(niz).replace(')(', ') (').replace('-', ' ').lower().split() if beseda
    }))

In [124]:
opis = """Whenever another creature enters the battlefield under your control, you gain life equal to that creature's toughness.
(1)(g)(w), (tap): Populate. (Put a token onto the battlefield that's a copy of a creature token you control.)"""
koreni_besed(opis)

0             (1)
1             (g)
2            (put
3          (tap):
4            (w),
5               @
6         another
7     battlefield
8        control,
9       control.)
10           copy
11        creatur
12      creature'
13          enter
14          equal
15           gain
16            lif
17             of
18            ont
19      populate.
20              t
21             th
22           that
23          that'
24            tok
25     toughness.
26          under
27       whenever
28              y
29           your
dtype: object

In [125]:
razpredelnica_s_koreni_oracle_textov = podatkovna_baza_kart.oracle_text.apply(koreni_besed)
razpredelnica_s_koreni_oracle_textov_brez_korenov = podatkovna_baza_kart.oracle_text.apply(koreni_besed, ne_naredi_korenov = True)
print(razpredelnica_s_koreni_oracle_textov)

           0             1           2        3          4           5   \
0        (x).             @  controller  counter       exil  graveyard.   
1           @      addition         and   beacon       bolt          by   
2       +2/+2           and          at  attack,  battalion         bor   
3           @  battlefield,        card   cards.      enter        from   
4           2            an        cast    damag       deal        each   
...       ...           ...         ...      ...        ...         ...   
1995     (1).             1           @  creatur      dies,         do,   
1996     (u).            at      beginn      fly       forc          of   
1997     (r):         +1/+0     attack.    can't    creatur    defender   
1998  creatur         hand.          it   owner'     return           t   
1999  creatur       doesn't      effect  enchant       from           h   

              6           7          8         9   ...   58   59   60   61  \
0             if     

In [126]:
koreni_tekstov = (
    razpredelnica_s_koreni_oracle_textov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ["level_1", 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1    # stolpcema damo smiselni imeni                            # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)

koreni_tekstov_brez_korenov = (
    razpredelnica_s_koreni_oracle_textov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ["level_1", 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1    # stolpcema damo smiselni imeni                            # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)

koreni_tekstov

,indeks_oracle_teksta,koren
0,0,(x).
1,1,@
2,2,+2/+2
3,3,@
4,4,2
...,...,...
127015,1015,with
129015,1015,y
131015,1015,your
133015,1015,−3:


In [127]:
#
tabela_barv = podatkovna_baza_kart[["barva"]]
tabela_barv.index.name = "indeks_oracle_teksta"
#print(koreni_tekstov, tabela_barv)

skupna_tabela = pd.merge(koreni_tekstov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
skupna_tabela_brez_korenov = pd.merge(koreni_tekstov_brez_korenov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
print(skupna_tabela)

       indeks_oracle_teksta       koren barva
0                         0        (x).     u
1                         0           @     u
2                         0  controller     u
3                         0     counter     u
4                         0        exil     u
...                     ...         ...   ...
42163                  1999       remov     w
42164                  1999          th     w
42165                  1999       ward.     w
42166                  1999        whit     w
42167                  1999      white.     w

[42168 rows x 3 columns]


In [128]:
stolpec_verjetnosti_korenov_pri_oracle_tekstu = skupna_tabela.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu = stolpec_verjetnosti_korenov_pri_oracle_tekstu.unstack().transpose().fillna(1 / len(podatkovna_baza_kart))
stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = skupna_tabela_brez_korenov.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.unstack().transpose().fillna(1 / len(podatkovna_baza_kart))
#verjetnosti_korenov_pri_oracle_tekstu
print(verjetnosti_korenov_pri_oracle_tekstu)

barva         b        bg     bgu      br     brg         c         g  \
koren                                                                   
"(1):  0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
"(2)   0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
"(2),  0.013652  0.083333  0.0005  0.0005  0.0005  0.006645  0.021583   
"(2):  0.003413  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
"(3)   0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
...         ...       ...     ...     ...     ...       ...       ...   
−6:    0.003413  0.041667  0.0005  0.0005  0.0005  0.000500  0.000500   
−7:    0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
−8:    0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.003597   
−9:    0.003413  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   
−x:    0.000500  0.000500  0.0005  0.0005  0.0005  0.000500  0.000500   

barva        gu     gur      gw  ...      ug      

In [129]:
def ugani_barvo(opis, ne_naredi_korenov = False):
    koreni_oracle_teksta = koreni_besed(opis, ne_naredi_korenov=ne_naredi_korenov)
    if ne_naredi_korenov:
        r = verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.reset_index()
    else:
        r = verjetnosti_korenov_pri_oracle_tekstu.reset_index()
    verjetnosti = r[r.koren.isin(koreni_oracle_teksta)].product(numeric_only=True) * verjetnosti_barve
    return verjetnosti.sort_values(ascending=False).head(10)


poskusni_tekst = """
Flying
Whenever Abyssal Specter deals damage to a player, that player discards a card.
"""

print("Če ustvarimo korene:")
print(ugani_barvo(poskusni_tekst))
print("Če ne ustvarimo korenov:")
print(ugani_barvo(poskusni_tekst, ne_naredi_korenov=True))




Če ustvarimo korene:
barva
c     4.216041e-53
u     8.733012e-58
g     4.661098e-61
ub    2.624705e-61
b     6.632134e-62
r     2.111973e-62
w     2.702671e-63
bg    2.745753e-67
gu    7.231209e-71
ur    2.694690e-72
dtype: float64
Če ne ustvarimo korenov:
barva
c     3.866277e-41
u     7.917045e-46
g     7.260046e-49
b     3.597337e-49
r     1.695607e-49
w     1.265670e-49
ub    1.979750e-50
rg    1.075489e-52
bg    9.671669e-55
gu    2.078843e-55
dtype: float64
